<a href="https://colab.research.google.com/github/Abinaya-hub2006/Computer_Vision/blob/main/Background_Subtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np

class BackgroundSubtractor:
    def __init__(self, alpha=0.05, scene_change_threshold=0.3):
        self.alpha = alpha  # Learning rate for background averaging
        self.scene_change_threshold = scene_change_threshold
        self.background = None  # Background model
        self.previous_frame = None

    def apply(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)

        if self.background is None:
            self.background = gray.astype(np.float32)
            return np.zeros_like(gray, dtype=np.uint8)

        # Update background model
        cv2.accumulateWeighted(gray, self.background, self.alpha)
        background_frame = cv2.convertScaleAbs(self.background)

        # Compute foreground mask
        foreground_mask = cv2.absdiff(gray, background_frame)
        _, thresh = cv2.threshold(foreground_mask, 30, 255, cv2.THRESH_BINARY)

        return thresh

# Load input video
cap = cv2.VideoCapture("/content/2053100-uhd_3840_2160_30fps.mp4")

# Get video properties
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define video writers
fourcc = cv2.VideoWriter_fourcc(*'XVID')
original_out = cv2.VideoWriter("original_output.avi", fourcc, fps, (frame_width, frame_height))
subtracted_out = cv2.VideoWriter("bg_subtracted_output.avi", fourcc, fps, (frame_width, frame_height), isColor=False)

background_subtractor = BackgroundSubtractor()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    fg_mask = background_subtractor.apply(frame)

    # Save frames to videos
    original_out.write(frame)
    subtracted_out.write(fg_mask)

cap.release()
original_out.release()
subtracted_out.release()

print("Videos saved: original_output.avi and bg_subtracted_output.avi")



Videos saved: original_output.avi and bg_subtracted_output.avi
